In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import math
from datetime import datetime 
import hashlib
import time
from email.message import EmailMessage
import sys
import os
path = os.getcwd()
pathing = sys.path.append(path)
sys.path.append(os.path.abspath('.')) 

# from internal_modules.email import send_email
# from internal_modules.email import sent

In [7]:
aces_individual = 'https://www.ncaa.com/stats/volleyball-men/d1/current/individual/532'

In [10]:
service = requests.get(aces_individual)

In [11]:
collecting = service.content
information_aces = BeautifulSoup(collecting, 'html.parser')
information_aces.prettify()

date_updated = '<em class="placeholder">Monday, January 27, 2025 8:08 am</em>'

In [12]:
def last_update():
    try:
        url_aces = 'https://www.ncaa.com/stats/volleyball-men/d1/current/individual/532'
        service = requests.get(url_aces)
        collecting = service.content
        information_aces = BeautifulSoup(collecting, 'html.parser')
        most_recent_date = information_aces.find_all('em')[1].text.strip()
        if len(most_recent_date) <= 1:
            return f'There has been an issue with the retrieving of the data. Contact the Author to resolve the issue.'
        else:
            time_stamp = datetime.strptime(most_recent_date, "%A, %B %d, %Y %I:%M %p")  
            return time_stamp
    except Exception as error:
        return f'There was an error at {error}, solve it to be able to parse the last updated date.'

updated = last_update()

def checking_update():
    last_known_update = updated  
    if not last_known_update:
        print(f'There has been an issue retrieving the last known update. Closing.')
        return None
    print(f'Process to check if there was an update has begun.')

    while True:
        try:
            start_time = time.time()
            
            time.sleep(604800)  
            
            print("Fetching current update timestamp...")
            current_update = updated
            
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if not current_update:
                print(f"Failed to fetch the current update time. Elapsed time: {elapsed_time:.2f} seconds. Retrying...")
            if current_update != last_known_update:
                print(f"The monitored part of the website has been updated! Elapsed time: {elapsed_time:.2f} seconds.")
                last_known_update = current_update
                break
            else:
                print(f"No update detected. Still monitoring. Elapsed time: {elapsed_time:.2f} seconds.")
        except KeyboardInterrupt:
            print("Monitoring stopped.")
        except Exception as e:
            print(f"An error occurred during monitoring: {e}")
            

            

In [118]:
last_updated = information_aces.find_all('em')[0]
# this is the going to be the date of when the page is updated
most_recent_date = information_aces.find_all('em')[1].text.strip()
most_recent_date
time_stamp = datetime.strptime(most_recent_date, "%A, %B %d, %Y %I:%M %p") 
print(time_stamp)


2025-01-31 08:05:00


In [120]:
# in order to make the data frame we want to make, we will need to find all the headers and columns we were using earlier
columns_aces = information_aces.find('tr').text
columns_aces = columns_aces.replace('\n',' ').strip().split()
df = pd.DataFrame(columns = columns_aces)
df['Per Set'] = df['Per'].astype(str) + df['Set'].astype(str)
df = df.drop(['Per','Set'], axis = 1)

In [128]:
# putting everything all together

table_data = information_aces.find_all('tr')

valid_ranking = 0

for item in table_data[1:]:
    vb = item.find_all('td')
    pieced = [info.text.strip() for info in vb]
    if pieced[0] == '-':
        valid_ranking += 1
        pieced[0] = str(valid_ranking)
    else:
        valid_ranking= int(pieced[0])
    length = len(df)
    df.loc[length] = pieced


# changing the data types
df['S'] = df['S'].astype(int)
df['Aces'] = df['Aces'].astype(int)
df['Per Set'] = df['Per Set'].astype(float)
df['Rank'] = df['Rank'].astype(int)


AttributeError: 'Series' object has no attribute 'type'

In [126]:
df

top_five_df = df[df['Team'].isin(['BYU', 'Hawaii','Long Beach St.', 'UCLA', 'UC Irvine'])]

top_five_df

freshies = top_five_df[top_five_df['Cl'].isin('Fr.')]

TypeError: only list-like objects are allowed to be passed to isin(), you passed a `str`

In [110]:
# we usually will use .3 to .5 for the best serving teams
# will research more on the 

average = df[(df['Per Set'] > .3) & (df['Per Set'] < .5) & (df['Aces'] >= 10) & (df['S'] > int(20))]
average

df

,Rank,Name,Team,Cl,Height,S,Aces,Per Set
0,1,Moni Nikolov,Long Beach St.,Fr.,6-8,21,22,1.048
1,2,Hilir Henno,UC Irvine,Sr.,6-8,17,17,1.000
2,3,Aleksa Mandic,Sacred Heart,So.,6-5,8,8,1.000
3,4,Kristian Titriyski,Hawaii,Fr.,6-5,18,15,0.833
4,5,Bryce Dvorak,UC San Diego,Sr.,6-4,26,21,0.808
...,...,...,...,...,...,...,...,...
195,46,Stanislaw Chacinski,Ohio St.,Fr.,6-7,20,8,0.400
196,47,Jackson Herbert,George Mason,So.,6-4,15,6,0.400
197,48,Griffin Monrad,Roberts Wesleyan,So.,6-3,25,10,0.400
198,49,Sawyer Nichols,Harvard,Fr.,6-9,10,4,0.400


In [11]:
# current status of project: 
# I am looking to do some research on the historcally best teams with their serving percentages and the overall 
# performance of the teams

# Automation should come next given the finalized of formulaic approach I decide to take

In [12]:
# next update, put key on gitignore, and include the internal module folder in git given
# it will be needed in the future of the project
